In [1]:
! pip install transformers
! pip install tensorflow==2.3.0
import tensorflow as tf
from transformers import RobertaConfig, AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
# import tensorflow as tf
tf.config.list_physical_devices('GPU')

     |████████████████████████████████| 3.1 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 126.1 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 14.7 MB/s  eta 0:00:01
     |████████████████████████████████| 895 kB 78.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.
     |███████████████████████████▉    | 278.6 MB 124.5 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 320.4 MB 43 kB/s 
     |████████████████████████████████| 20.1 MB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 120.0 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled scipy-1.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.4
    Uninstalling tensorflow-2.3.4:
      Successfully uninstalled tensorflow-2.3.4
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [81]:
! pip install xgboost
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import argparse
import logging
import os
import sys
import csv
import s3fs
import json
import re
import gc
import scipy.stats as st
from sklearn.metrics import precision_recall_curve


fs = s3fs.S3FileSystem()
MAX_LEN = 128
PEACE_COUNTRY = set(['Australia', 'New Zealand', 
                 'Belgium', 'Sweden', 'Denmark', 
                 'Norway', 'Finland', 'Czech Republic', 
                 'Netherlands', 'Austria'])
MAJOR_COUNTRY = set(['Australia', 'India'])

In [3]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
config = RobertaConfig.from_pretrained(
    'roberta-base',
    num_labels=1, #Binary Classification
    dropout=0.1,
    attention_dropout=0.1,
    output_hidden_states=False,
    output_attentions=False
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
# Define encoding function parser
def regular_encode(texts, tokenizer, maxlen=MAX_LEN):
    """
    Function to encode the word
    """
    # encode the word to vector of integer
    enc_di = tokenizer.encode_plus(
        texts, 
        return_attention_mask=True, 
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        max_length=maxlen)
    
    return np.array(enc_di['input_ids']), np.array(enc_di['attention_mask'])

## Load Unshuffled, Original Data

In [112]:
# Load in data
train_count = 0  
val_count = 0  
train_label_counter = [0, 0]
val_label_counter = [0, 0]

train_label_count_max = 4e4
val_label_count_max = 1e4
total_train = 2 * train_label_count_max
total_val = 2 * val_label_count_max

X_train = []
y_train = []

X_val = []
y_val = []

for line in fs.open('s3://compressed-data-sample/processed_train.json'):
    if train_count >= total_train and val_count >= total_val:
        break
    json_file = json.loads(line)
    country = json_file['country']
    label =  int(json_file['country'] in PEACE_COUNTRY)
    
    if not country in MAJOR_COUNTRY:
        if train_label_counter[label] < train_label_count_max :
            sent = json_file['content_cleaned']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_train.append({'input_ids': ids,'attention_mask':msk})
            y_train.append(label)
            train_count += 1
            train_label_counter[label] += 1
            if sum(train_label_counter) % 1e3 == 0:
                print('Train', train_label_counter)
    else:
        if val_label_counter[label] < val_label_count_max :
            sent = json_file['content_cleaned']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_val.append({'input_ids': ids,'attention_mask':msk})
            y_val.append(label)
            val_count += 1
            val_label_counter[label] += 1
            if sum(val_label_counter) % 1e3 == 0:
                print('Val', val_label_counter)

Val [532, 468]
Val [1065, 935]
Val [1607, 1393]
Val [2144, 1856]
Val [2690, 2310]
Train [240, 760]
Val [3255, 2745]
Val [3805, 3195]
Val [4348, 3652]
Val [4906, 4094]
Val [5457, 4543]
Train [481, 1519]
Val [5984, 5016]
Val [6519, 5481]
Val [7059, 5941]
Val [7638, 6362]
Val [8191, 6809]
Train [749, 2251]
Val [8747, 7253]
Val [9315, 7685]
Val [9837, 8163]
Val [10000, 9000]
Train [1028, 2972]
Val [10000, 10000]
Train [1268, 3732]
Train [1522, 4478]
Train [1770, 5230]
Train [2027, 5973]
Train [2301, 6699]
Train [2562, 7438]
Train [2796, 8204]
Train [3071, 8929]
Train [3327, 9673]
Train [3601, 10399]
Train [3863, 11137]
Train [4128, 11872]
Train [4384, 12616]
Train [4619, 13381]
Train [4861, 14139]
Train [5101, 14899]
Train [5352, 15648]
Train [5588, 16412]
Train [5843, 17157]
Train [6113, 17887]
Train [6353, 18647]
Train [6634, 19366]
Train [6926, 20074]
Train [7193, 20807]
Train [7450, 21550]
Train [7714, 22286]
Train [7969, 23031]
Train [8237, 23763]
Train [8478, 24522]
Train [8739, 2526

In [113]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)

X_all = np.hstack([X_train, X_val])
y_all = np.hstack([y_train, y_val])

## Prepare for model training

In [8]:
from sklearn.model_selection import StratifiedKFold

def get_model(lr = 3e-5):
    bert_model = TFAutoModelForSequenceClassification.from_pretrained('roberta-base', trainable=True, config=config)
    input_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
    input_masks_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='attention_mask', dtype='int32')
    output_layer = bert_model(input_ids_in, input_masks_ids_in)[0]
    output_layer = tf.keras.layers.Activation(activation='sigmoid')(output_layer)
    model = tf.keras.Model(inputs=[input_ids_in, input_masks_ids_in], outputs = output_layer)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.5),
               tf.keras.metrics.Precision(name='precision', thresholds=0.5),
               tf.keras.metrics.Recall(name='recall', thresholds=0.5),
               tf.keras.metrics.TruePositives(name='TP', thresholds=0.5),
               tf.keras.metrics.TrueNegatives(name='TN', thresholds=0.5),
               tf.keras.metrics.FalsePositives(name='FP', thresholds=0.5),
               tf.keras.metrics.FalseNegatives(name='FN', thresholds=0.5)]
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

# Define Splits
skf_train = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
skf_val = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

## Define test runner

In [103]:
# Construct test function
def run_test(X_train, y_train, 
             X_val, y_val,
             train_index, val_index, 
             class_weight, epochs, batch_size=32, lr=3e-5):
    eval_results = []
    eval_results_xgb = []
    eval_results_svm = []
    metrics_names = None

    for train_index, val_index in list(zip(train_index, val_index)):
        tf.keras.backend.clear_session()
        
        X_train_fold, X_val_fold = X_train[train_index], X_val[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_val[val_index]
        y_train_fold = np.asarray(y_train_fold)
        y_val_fold = np.asarray(y_val_fold)

        model = get_model(lr)
        
        train_input1 = np.vstack([x['input_ids'] for x in X_train_fold])
        train_input2 = np.vstack([x['attention_mask'] for x in X_train_fold])
        model.fit(x=[train_input1, train_input2], 
                  y=y_train_fold,
                  epochs = epochs, 
                  batch_size = batch_size,
                  class_weight=class_weight)

        eval_input1 = np.vstack([x['input_ids'] for x in X_val_fold])
        eval_input2 = np.vstack([x['attention_mask'] for x in X_val_fold])
        er = model.evaluate(x=[eval_input1, eval_input2], 
                            y=y_val_fold, return_dict=True)
        f1 = 2*er['precision']*er['recall'] / (er['precision']+er['recall'])
        
        precision_neg = er['TN'] / (er['TN'] + er['FN'])
        recall_neg = er['TN'] / (er['TN']+er['FP'])
        f1_neg = 2 * precision_neg * recall_neg / (precision_neg + recall_neg)
        
        er = list(er.values())
        er += [f1, precision_neg, recall_neg, f1_neg]
        eval_results.append(er)
        metrics_names = model.metrics_names
        print(er)
        
        # Get Embeddings 
        input_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
        input_masks_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='attention_mask', dtype='int32')
        output_layer = model.layers[2].layers[0](input_ids_in, input_masks_ids_in)[0]
        embedding_model = tf.keras.Model(inputs=[input_ids_in, input_masks_ids_in], outputs = output_layer)
        
        train_embedding = embedding_model.predict(x=[train_input1, train_input2])[:, 0]
        val_embedding = embedding_model.predict(x=[eval_input1, eval_input2])[:, 0]
        
        # Get XGB results
        dtrain = xgb.DMatrix(train_embedding, label=y_train_fold)
        dval = xgb.DMatrix(val_embedding, label=y_val_fold)
        param = {'max_depth': 2, 'eta': 1,'objective': 'binary:logistic'}
        num_round = 10
        evallist = [(dval, 'eval'), (dtrain, 'train')]
        bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
        y_pred = (bst.predict(dval) > 0.5).astype(int)
        xgb_acc = np.mean(y_val_fold == y_pred)
        xgb_score = precision_recall_fscore_support(y_val_fold, y_pred)
        # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
        er = [xgb_acc, xgb_score[0][1], xgb_score[1][1], xgb_score[2][1], 
                                 xgb_score[0][0], xgb_score[1][0], xgb_score[2][0]]
        print(er)
        eval_results_xgb.append(er)
        
        
        # Get SVM results
        pipe = Pipeline([('scaler', StandardScaler()),
                         ('pca', PCA()),
                         ('svc', SVC())])
        pipe.fit(train_embedding, y_train_fold)
        y_pred = (pipe.predict(val_embedding) > 0.5).astype(int)
        svm_acc = np.mean(y_val_fold == y_pred)
        svm_score = precision_recall_fscore_support(y_val_fold, y_pred)
        # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
        er = [svm_acc, svm_score[0][1], svm_score[1][1], svm_score[2][1], 
                                 svm_score[0][0], svm_score[1][0], svm_score[2][0]]
        print(er)
        eval_results_svm.append(er)
        
        
        ## clear memory
        del model
        del X_train_fold, X_val_fold, y_train_fold, y_val_fold
        del train_input1, train_input2, eval_input1, eval_input2
        gc.collect() 

    
    eval_results = np.array(eval_results)
    metrics_names += ['f1', 'precision_neg', 'recall_neg', 'f1_neg']
    eval_results = pd.DataFrame(eval_results, columns=metrics_names)
    ci = eval_results.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
    ci.index = ['ci_lower', 'ci_upper']
    eval_results = pd.concat([eval_results.describe(), ci])
    
    eval_results_xgb = np.array(eval_results_xgb)
    eval_results_xgb = pd.DataFrame(eval_results_xgb, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
    ci = eval_results_xgb.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
    ci.index = ['ci_lower', 'ci_upper']
    eval_results_xgb = pd.concat([eval_results_xgb.describe(), ci])
    
    eval_results_svm = np.array(eval_results_svm)
    eval_results_svm = pd.DataFrame(eval_results_svm, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
    ci = eval_results_svm.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
    ci.index = ['ci_lower', 'ci_upper']
    eval_results_svm = pd.concat([eval_results_svm.describe(), ci])
    
    
    return eval_results, eval_results_xgb, eval_results_svm

### Random Split - Original

In [107]:
from sklearn.model_selection import train_test_split

# Throw out some data to make sample # same as split by country
_, train_index = train_test_split(np.array(range(len(X_all))),test_size=0.1, stratify=y_all)

train_index, val_index = list(zip(*[train_test_split(np.array(range(len(X_all))), 
                            test_size=0.2, stratify=y_all) for i in range(10)]))

test_result, test_result_xgb, test_result_svm = run_test(X_all, y_all, X_all, y_all, 
                                                         train_index,val_index,class_weight = {0: 1., 1: 1.}, epochs=1)
print(test_result)
print(test_result_xgb)
print(test_result_svm)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 261ms/step - loss: 0.1331 - accuracy: 0.9540 - precision: 0.9809 - recall: 0.9260 - TP: 926.0000 - TN: 982.0000 - FP: 18.0000 - FN: 74.0000
[0.13308829069137573, 0.9539999961853027, 0.9809321761131287, 0.9259999990463257, 926.0, 982.0, 18.0, 74.0, 0.9526748837507167, 0.9299242424242424, 0.982, 0.9552529182879378]
[22:43:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9475, 0.9470529470529471, 0.948, 0.9475262368815592, 0.9479479479479479, 0.947, 0.9474737368684342]
[0.9585, 0.9598796389167502, 0.957, 0.958437656484727, 0.9571286141575274, 0.96, 0.9585621567648526]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 255ms/step - loss: 0.1488 - accuracy: 0.9370 - precision: 0.9740 - recall: 0.8980 - TP: 898.0000 - TN: 976.0000 - FP: 24.0000 - FN: 102.0000
[0.14880239963531494, 0.9369999766349792, 0.9739696383476257, 0.8980000019073486, 898.0, 976.0, 24.0, 102.0, 0.9344432925469219, 0.9053803339517625, 0.976, 0.9393647738209817]
[22:49:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9455, 0.953204476093591, 0.937, 0.9450327786182552, 0.9380530973451328, 0.954, 0.945959345562717]
[0.952, 0.9510978043912176, 0.953, 0.952047952047952, 0.9529058116232465, 0.951, 0.951951951951952]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

63/63 [==============================] - 15s 237ms/step - loss: 0.1735 - accuracy: 0.9340 - precision: 0.9687 - recall: 0.8970 - TP: 897.0000 - TN: 971.0000 - FP: 29.0000 - FN: 103.0000
[0.17351080477237701, 0.9340000152587891, 0.9686825275421143, 0.8970000147819519, 897.0, 971.0, 29.0, 103.0, 0.9314641926615262, 0.904096834264432, 0.971, 0.9363548698167793]
[23:22:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.945, 0.9405940594059405, 0.95, 0.9452736318407959, 0.9494949494949495, 0.94, 0.9447236180904524]
[0.949, 0.9472111553784861, 0.951, 0.9491017964071856, 0.9508032128514057, 0.947, 0.9488977955911825]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 257ms/step - loss: 0.1354 - accuracy: 0.9490 - precision: 0.9563 - recall: 0.9410 - TP: 941.0000 - TN: 957.0000 - FP: 43.0000 - FN: 59.0000
[0.13537751138210297, 0.9490000009536743, 0.9563007950782776, 0.9409999847412109, 941.0, 957.0, 43.0, 59.0, 0.9485886931035602, 0.9419291338582677, 0.957, 0.9494047619047619]
[23:28:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9545, 0.9513406156901688, 0.958, 0.9546586945690084, 0.9577039274924471, 0.951, 0.9543401906673357]
[0.957, 0.9489194499017681, 0.966, 0.9573835480673935, 0.9653767820773931, 0.948, 0.9566094853683148]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 15s 237ms/step - loss: 0.1581 - accuracy: 0.9450 - precision: 0.9754 - recall: 0.9130 - TP: 913.0000 - TN: 977.0000 - FP: 23.0000 - FN: 87.0000
[0.15811298787593842, 0.9449999928474426, 0.9754273295402527, 0.9129999876022339, 913.0, 977.0, 23.0, 87.0, 0.9431818018018381, 0.918233082706767, 0.977, 0.9467054263565892]
[23:33:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.952, 0.9565656565656566, 0.947, 0.9517587939698493, 0.9475247524752475, 0.957, 0.9522388059701493]
[0.9545, 0.9577039274924471, 0.951, 0.9543401906673357, 0.9513406156901688, 0.958, 0.9546586945690084]
               loss   accuracy  precision     recall          TP          TN  \
count     10.000000  10.000000  10.000000  10.000000   10.000000   10.000000   
mean       0.150779   0.

### Split by country - Original

In [114]:
train_index = [val_split for _, val_split in skf_train.split(X_train,y_train)] 
val_index = [val_split for _, val_split in skf_val.split(X_val,y_val)] 

test_result, test_result_xgb, test_result_svm = run_test(X_train, y_train, X_val, y_val,
                       train_index, val_index, class_weight = {0: 1., 1: 1.}, epochs=1)
print(test_result)
print(test_result_xgb)
print(test_result_svm)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 262ms/step - loss: 0.9802 - accuracy: 0.6765 - precision: 0.6544 - recall: 0.7480 - TP: 748.0000 - TN: 605.0000 - FP: 395.0000 - FN: 252.0000
[0.9802479147911072, 0.6765000224113464, 0.6544181704521179, 0.7480000257492065, 748.0, 605.0, 395.0, 252.0, 0.6980867899102214, 0.705950991831972, 0.605, 0.6515885837372105]
[01:14:32] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.679, 0.6559233449477352, 0.753, 0.7011173184357542, 0.710093896713615, 0.605, 0.6533477321814255]
[0.677, 0.6523235800344234, 0.758, 0.7012025901942646, 0.711217183770883, 0.596, 0.6485310119695321]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 255ms/step - loss: 0.9103 - accuracy: 0.6535 - precision: 0.6795 - recall: 0.5810 - TP: 581.0000 - TN: 726.0000 - FP: 274.0000 - FN: 419.0000
[0.9102622866630554, 0.6535000205039978, 0.6795321702957153, 0.5809999704360962, 581.0, 726.0, 274.0, 419.0, 0.6264150799407273, 0.634061135371179, 0.726, 0.6769230769230768]
[01:20:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.714, 0.6739837398373983, 0.829, 0.7434977578475335, 0.7779220779220779, 0.599, 0.6768361581920904]
[0.721, 0.6823432343234324, 0.827, 0.7477396021699819, 0.7804568527918782, 0.615, 0.6879194630872483]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

63/63 [==============================] - 16s 255ms/step - loss: 0.9637 - accuracy: 0.6630 - precision: 0.6760 - recall: 0.6260 - TP: 626.0000 - TN: 700.0000 - FP: 300.0000 - FN: 374.0000
[0.9636971354484558, 0.6629999876022339, 0.676025927066803, 0.6259999871253967, 626.0, 700.0, 300.0, 374.0, 0.6500519183641734, 0.6517690875232774, 0.7, 0.6750241080038571]
[01:53:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.729, 0.6805993690851735, 0.863, 0.7610229276895942, 0.8128415300546448, 0.595, 0.6870669745958429]
[0.736, 0.6900161030595813, 0.857, 0.7644959857270294, 0.8113456464379947, 0.615, 0.6996587030716723]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 257ms/step - loss: 0.9279 - accuracy: 0.7195 - precision: 0.6644 - recall: 0.8870 - TP: 887.0000 - TN: 552.0000 - FP: 448.0000 - FN: 113.0000
[0.9279272556304932, 0.7195000052452087, 0.6644194722175598, 0.8870000243186951, 887.0, 552.0, 448.0, 113.0, 0.7597430473583299, 0.8300751879699249, 0.552, 0.6630630630630631]
[01:59:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.7245, 0.6794564348521183, 0.85, 0.7552199022656596, 0.7997329773030708, 0.599, 0.6849628359062321]
[0.7285, 0.6853203568532036, 0.845, 0.7568293775190327, 0.7979139504563233, 0.612, 0.6926994906621391]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 256ms/step - loss: 0.8733 - accuracy: 0.6950 - precision: 0.6653 - recall: 0.7850 - TP: 785.0000 - TN: 605.0000 - FP: 395.0000 - FN: 215.0000
[0.8733049035072327, 0.6949999928474426, 0.6652542352676392, 0.7850000262260437, 785.0, 605.0, 395.0, 215.0, 0.7201834960915343, 0.7378048780487805, 0.605, 0.6648351648351648]
[02:04:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.701, 0.6680602006688964, 0.799, 0.7276867030965392, 0.75, 0.603, 0.6685144124168514]
[0.702, 0.6717687074829932, 0.79, 0.7261029411764707, 0.7451456310679612, 0.614, 0.6732456140350876]
               loss   accuracy  precision     recall          TP          TN  \
count     10.000000  10.000000  10.000000  10.000000   10.000000   10.000000   
mean       0.951950   0.690700   0.6

### Balanced Random Split - Original

In [126]:
# Load in data
train_count = 0  
val_count = 0  
train_label_counter = [0, 0]
val_label_counter = [0, 0]

train_label_count_max = 5e3 
val_label_count_max = 1e3
total_train = 2 * train_label_count_max
total_val = 2 * val_label_count_max

X_train = []
y_train = []

X_val = []
y_val = []

for line in fs.open('s3://compressed-data-sample/processed_train.json'):
    if train_count >= total_train and val_count >= total_val:
        break
    json_file = json.loads(line)
    country = json_file['country']
    label =  int(json_file['country'] in PEACE_COUNTRY)
    
    if not country in MAJOR_COUNTRY:
        if train_label_counter[label] < train_label_count_max :
            sent = json_file['content_cleaned']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_train.append({'input_ids': ids,'attention_mask':msk})
            y_train.append(json_file['country'])
            train_count += 1
            train_label_counter[label] += 1
            if sum(train_label_counter) % 1e3 == 0:
                print('Train', train_label_counter)
    else:
        if val_label_counter[label] < val_label_count_max :
            sent = json_file['content_cleaned']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_val.append({'input_ids': ids,'attention_mask':msk})
            y_val.append(json_file['country'])
            val_count += 1
            val_label_counter[label] += 1
            if sum(val_label_counter) % 1e3 == 0:
                print('Val', val_label_counter)

Val [532, 468]
Val [1000, 1000]
Train [240, 760]
Train [481, 1519]
Train [749, 2251]
Train [1028, 2972]
Train [1268, 3732]
Train [1522, 4478]
Train [2000, 5000]
Train [3000, 5000]
Train [4000, 5000]
Train [5000, 5000]


In [127]:
import statistics
from collections import Counter
median = statistics.median(list(Counter(y_train).values()))
median

287

In [128]:
## Prepare K-Fold for minority country
import random
X_train = np.array(X_train)
X_val = np.array(X_val)

y_train = np.array(y_train)
y_val = np.array(y_val)

australia_idx = np.where(np.array(y_val) == 'Australia')[0]
india_idx = np.where(np.array(y_val) == 'India')[0]
majority_index = []
for i in range(10):
    australia_sample = np.array(random.sample(list(australia_idx), median))
    india_sample = np.array(random.sample(list(india_idx), median))
    majority_index.append(np.hstack([australia_sample, india_sample])) 

In [129]:
eval_results = []
eval_results_xgb = []
eval_results_svm = []
metrics_names = None
    
for maj_index in majority_index:
    tf.keras.backend.clear_session()
    X_all_sample = np.hstack([X_train, X_val[maj_index]])
    y_all_sample = np.hstack([y_train, y_val[maj_index]])
    y_all_sample = np.array(list(map(lambda x: int(x in PEACE_COUNTRY), y_all_sample)))
    
    X_train_fold, X_val_fold, y_train_fold, y_val_fold = train_test_split(X_all_sample, y_all_sample, test_size=0.2)
    
    model = get_model()
        
    train_input1 = np.vstack([x['input_ids'] for x in X_train_fold])
    train_input2 = np.vstack([x['attention_mask'] for x in X_train_fold])
    model.fit(x=[train_input1, train_input2], 
              y=np.asarray(y_train_fold),
              epochs = 1, 
              batch_size = 32,
              class_weight={0: 1., 1: 1.})

    eval_input1 = np.vstack([x['input_ids'] for x in X_val_fold])
    eval_input2 = np.vstack([x['attention_mask'] for x in X_val_fold])
    er = model.evaluate(x=[eval_input1, eval_input2], 
                        y=np.asarray(y_val_fold), return_dict=True)
    f1 = 2*er['precision']*er['recall'] / (er['precision']+er['recall'])

    precision_neg = er['TN'] / (er['TN'] + er['FN'])
    recall_neg = er['TN'] / (er['TN']+er['FP'])
    f1_neg = 2 * precision_neg * recall_neg / (precision_neg + recall_neg)

    er = list(er.values())
    er += [f1, precision_neg, recall_neg, f1_neg]
    eval_results.append(er)
    metrics_names = model.metrics_names
    print(er)

    # Get Embeddings 
    input_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
    input_masks_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='attention_mask', dtype='int32')
    output_layer = model.layers[2].layers[0](input_ids_in, input_masks_ids_in)[0]
    embedding_model = tf.keras.Model(inputs=[input_ids_in, input_masks_ids_in], outputs = output_layer)

    train_embedding = embedding_model.predict(x=[train_input1, train_input2])[:, 0]
    val_embedding = embedding_model.predict(x=[eval_input1, eval_input2])[:, 0]

    # Get XGB results
    dtrain = xgb.DMatrix(train_embedding, label=y_train_fold)
    dval = xgb.DMatrix(val_embedding, label=y_val_fold)
    param = {'max_depth': 2, 'eta': 1,'objective': 'binary:logistic'}
    num_round = 10
    evallist = [(dval, 'eval'), (dtrain, 'train')]
    bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
    y_pred = (bst.predict(dval) > 0.5).astype(int)
    xgb_acc = np.mean(y_val_fold == y_pred)
    xgb_score = precision_recall_fscore_support(y_val_fold, y_pred)
    # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
    er = [xgb_acc, xgb_score[0][1], xgb_score[1][1], xgb_score[2][1], 
                             xgb_score[0][0], xgb_score[1][0], xgb_score[2][0]]
    print(er)
    eval_results_xgb.append(er)


    # Get SVM results
    pipe = Pipeline([('scaler', StandardScaler()),
                     ('pca', PCA()),
                     ('svc', SVC())])
    pipe.fit(train_embedding, y_train_fold)
    y_pred = (pipe.predict(val_embedding) > 0.5).astype(int)
    svm_acc = np.mean(y_val_fold == y_pred)
    svm_score = precision_recall_fscore_support(y_val_fold, y_pred)
    # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
    er = [svm_acc, svm_score[0][1], svm_score[1][1], svm_score[2][1], 
                             svm_score[0][0], svm_score[1][0], svm_score[2][0]]
    print(er)
    eval_results_svm.append(er)
    
eval_results = np.array(eval_results)
metrics_names += ['f1', 'precision_neg', 'recall_neg', 'f1_neg']
eval_results = pd.DataFrame(eval_results, columns=metrics_names)
ci = eval_results.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results = pd.concat([eval_results.describe(), ci])

eval_results_xgb = np.array(eval_results_xgb)
eval_results_xgb = pd.DataFrame(eval_results_xgb, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
ci = eval_results_xgb.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results_xgb = pd.concat([eval_results_xgb.describe(), ci])

eval_results_svm = np.array(eval_results_svm)
eval_results_svm = pd.DataFrame(eval_results_svm, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
ci = eval_results_svm.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results_svm = pd.concat([eval_results_svm.describe(), ci])

eval_results, eval_results_xgb, eval_results_svm

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 257ms/step - loss: 0.1426 - accuracy: 0.9456 - precision: 0.9826 - recall: 0.9075 - TP: 962.0000 - TN: 1038.0000 - FP: 17.0000 - FN: 98.0000
[0.14255183935165405, 0.9456264972686768, 0.9826353192329407, 0.9075471758842468, 962.0, 1038.0, 17.0, 98.0, 0.9435997965251304, 0.9137323943661971, 0.9838862559241706, 0.947512551346417]
[05:02:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9555555555555556, 0.9530956848030019, 0.9584905660377359, 0.9557855126999059, 0.9580552907530981, 0.95260663507109, 0.9553231939163498]
[0.9612293144208038, 0.9561567164179104, 0.9669811320754716, 0.9615384615384615, 0.9664429530201343, 0.9554502369668246, 0.9609151572926597]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 258ms/step - loss: 0.1204 - accuracy: 0.9527 - precision: 0.9414 - recall: 0.9689 - TP: 1060.0000 - TN: 955.0000 - FP: 66.0000 - FN: 34.0000
[0.12041790783405304, 0.9527186751365662, 0.9413854479789734, 0.9689213633537292, 1060.0, 955.0, 66.0, 34.0, 0.9549549488972534, 0.9656218402426694, 0.9353574926542605, 0.9502487562189055]
[05:08:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9508274231678487, 0.9541284403669725, 0.9506398537477148, 0.9523809523809523, 0.9473170731707317, 0.951028403525955, 0.949169110459433]
[0.9602836879432625, 0.9658671586715867, 0.9570383912248629, 0.9614325068870524, 0.9544131910766246, 0.9637610186092067, 0.9590643274853801]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 254ms/step - loss: 0.1443 - accuracy: 0.9409 - precision: 0.9777 - recall: 0.9036 - TP: 966.0000 - TN: 1024.0000 - FP: 22.0000 - FN: 103.0000
[0.1442503035068512, 0.9408983588218689, 0.97773277759552, 0.9036482572555542, 966.0, 1024.0, 22.0, 103.0, 0.9392318771893668, 0.9086069210292813, 0.9789674952198852, 0.9424758398527382]
[05:50:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.956501182033097, 0.9710703953712633, 0.9420018709073901, 0.9563152896486229, 0.9424860853432282, 0.97131931166348, 0.9566854990583804]
[0.9593380614657211, 0.9667616334283001, 0.9522918615528532, 0.9594721960414704, 0.9519774011299436, 0.9665391969407265, 0.9592030360531308]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 253ms/step - loss: 0.1091 - accuracy: 0.9551 - precision: 0.9555 - recall: 0.9546 - TP: 1009.0000 - TN: 1011.0000 - FP: 47.0000 - FN: 48.0000
[0.10908699780702591, 0.9550827145576477, 0.9554924368858337, 0.9545884728431702, 1009.0, 1011.0, 47.0, 48.0, 0.9550402409596913, 0.9546742209631728, 0.9555765595463138, 0.9551251771374586]
[05:56:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9621749408983451, 0.9692603266090298, 0.9545884578997161, 0.9618684461391802, 0.9553072625698324, 0.9697542533081286, 0.9624765478424014]
[0.9631205673758865, 0.9622285174693107, 0.9640491958372753, 0.9631379962192818, 0.9640151515151515, 0.9621928166351607, 0.9631031220435194]


(               loss   accuracy  precision     recall           TP  \
 count     10.000000  10.000000  10.000000  10.000000    10.000000   
 mean       0.134713   0.949504   0.951683   0.948287  1004.200000   
 std        0.018017   0.005589   0.021227   0.027281    35.282353   
 min        0.109087   0.940898   0.915136   0.903648   961.000000   
 25%        0.116970   0.945745   0.943055   0.933904   971.750000   
 50%        0.141770   0.948463   0.954611   0.957332  1006.500000   
 75%        0.148428   0.954492   0.960823   0.967805  1028.750000   
 max        0.156319   0.956974   0.982635   0.980319  1060.000000   
 ci_lower   0.122018   0.945565   0.936726   0.929065   979.340076   
 ci_upper   0.147408   0.953442   0.966639   0.967509  1029.059924   
 
                    TN         FP          FN         f1  precision_neg  \
 count       10.000000  10.000000   10.000000  10.000000      10.000000   
 mean      1004.000000  52.100000   54.700000   0.949443       0.949444   
 st

## Shuffled Data

In [115]:
# Load in data
train_count = 0  
val_count = 0  
train_label_counter = [0, 0]
val_label_counter = [0, 0]

train_label_count_max = 4e4
val_label_count_max = 1e4
total_train = 2 * train_label_count_max
total_val = 2 * val_label_count_max

X_train = []
y_train = []

X_val = []
y_val = []

for line in fs.open('s3://compressed-data-sample/shuffled_train.json'):
    if train_count >= total_train and val_count >= total_val:
        break
    json_file = json.loads(line)
    country = json_file['country']
    label =  int(json_file['country'] in PEACE_COUNTRY)
    
    if not country in MAJOR_COUNTRY:
        if train_label_counter[label] < train_label_count_max :
            sent = json_file['content_cleaned_shuffled']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_train.append({'input_ids': ids,'attention_mask':msk})
            y_train.append(label)
            train_count += 1
            train_label_counter[label] += 1
            if sum(train_label_counter) % 1e3 == 0:
                print('Train', train_label_counter)
    else:
        if val_label_counter[label] < val_label_count_max :
            sent = json_file['content_cleaned_shuffled']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_val.append({'input_ids': ids,'attention_mask':msk})
            y_val.append(label)
            val_count += 1
            val_label_counter[label] += 1
            if sum(val_label_counter) % 1e3 == 0:
                print('Val', val_label_counter)

Val [532, 468]
Val [1065, 935]
Val [1607, 1393]
Val [2144, 1856]
Val [2690, 2310]
Train [240, 760]
Val [3255, 2745]
Val [3805, 3195]
Val [4348, 3652]
Val [4906, 4094]
Val [5457, 4543]
Train [481, 1519]
Val [5984, 5016]
Val [6519, 5481]
Val [7059, 5941]
Val [7638, 6362]
Val [8191, 6809]
Train [749, 2251]
Val [8747, 7253]
Val [9315, 7685]
Val [9837, 8163]
Val [10000, 9000]
Train [1028, 2972]
Val [10000, 10000]
Train [1268, 3732]
Train [1522, 4478]
Train [1770, 5230]
Train [2027, 5973]
Train [2301, 6699]
Train [2562, 7438]
Train [2796, 8204]
Train [3071, 8929]
Train [3327, 9673]
Train [3601, 10399]
Train [3863, 11137]
Train [4128, 11872]
Train [4384, 12616]
Train [4619, 13381]
Train [4861, 14139]
Train [5101, 14899]
Train [5352, 15648]
Train [5588, 16412]
Train [5843, 17157]
Train [6113, 17887]
Train [6353, 18647]
Train [6634, 19366]
Train [6926, 20074]
Train [7193, 20807]
Train [7450, 21550]
Train [7714, 22286]
Train [7969, 23031]
Train [8237, 23763]
Train [8478, 24522]
Train [8739, 2526

In [116]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)

X_all = np.hstack([X_train, X_val])
y_all = np.hstack([y_train, y_val])

### Random Split - Shuffled

In [111]:
from sklearn.model_selection import train_test_split

# Throw out some data to make sample # same as split by country
_, train_index = train_test_split(np.array(range(len(X_all))),test_size=0.1, stratify=y_all)

train_index, val_index = list(zip(*[train_test_split(np.array(range(len(X_all))), 
                            test_size=0.2, stratify=y_all) for i in range(10)]))

test_result, test_result_xgb, test_result_svm = run_test(X_all, y_all, X_all, y_all, 
                                                         train_index,val_index,class_weight = {0: 1., 1: 1.}, epochs=1)
print(test_result)
print(test_result_xgb)
print(test_result_svm)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 259ms/step - loss: 0.2040 - accuracy: 0.9165 - precision: 0.9016 - recall: 0.9350 - TP: 935.0000 - TN: 898.0000 - FP: 102.0000 - FN: 65.0000
[0.20400559902191162, 0.9164999723434448, 0.9016393423080444, 0.9350000023841858, 935.0, 898.0, 102.0, 65.0, 0.9180166913488028, 0.9325025960539979, 0.898, 0.9149261334691798]
[23:44:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9095, 0.9107321965897693, 0.908, 0.9093640460691037, 0.9082751744765702, 0.911, 0.90963554667998]
[0.925, 0.9267068273092369, 0.923, 0.9248496993987976, 0.9233067729083665, 0.927, 0.9251497005988024]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 15s 238ms/step - loss: 0.2893 - accuracy: 0.8910 - precision: 0.9278 - recall: 0.8480 - TP: 848.0000 - TN: 934.0000 - FP: 66.0000 - FN: 152.0000
[0.28933852910995483, 0.890999972820282, 0.9277899265289307, 0.8479999899864197, 848.0, 934.0, 66.0, 152.0, 0.8861023943078947, 0.860036832412523, 0.934, 0.8954937679769894]
[23:49:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.906, 0.9084507042253521, 0.903, 0.9057171514543632, 0.9035785288270378, 0.909, 0.9062811565304087]
[0.912, 0.9071146245059288, 0.918, 0.9125248508946322, 0.917004048582996, 0.906, 0.9114688128772637]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

63/63 [==============================] - 16s 259ms/step - loss: 0.2330 - accuracy: 0.8960 - precision: 0.8468 - recall: 0.9670 - TP: 967.0000 - TN: 825.0000 - FP: 175.0000 - FN: 33.0000
[0.23296888172626495, 0.8960000276565552, 0.846760094165802, 0.9670000076293945, 967.0, 825.0, 175.0, 33.0, 0.9028945081636329, 0.9615384615384616, 0.825, 0.8880516684607105]
[00:23:15] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9125, 0.9112662013958126, 0.914, 0.9126310534198703, 0.9137412236710131, 0.911, 0.9123685528292439]
[0.915, 0.9076620825147348, 0.924, 0.9157581764122895, 0.9226069246435845, 0.906, 0.9142280524722503]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 260ms/step - loss: 0.2147 - accuracy: 0.9115 - precision: 0.8677 - recall: 0.9710 - TP: 971.0000 - TN: 852.0000 - FP: 148.0000 - FN: 29.0000
[0.21465188264846802, 0.9114999771118164, 0.8677390813827515, 0.9710000157356262, 971.0, 852.0, 148.0, 29.0, 0.9164700560264692, 0.9670828603859251, 0.852, 0.9059011164274322]
[00:28:55] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9215, 0.9120234604105572, 0.933, 0.9223924864063273, 0.9314227226202662, 0.91, 0.9205867475973697]
[0.9275, 0.9138431752178122, 0.944, 0.9286768322675848, 0.9420889348500517, 0.911, 0.9262836807320795]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 256ms/step - loss: 0.2851 - accuracy: 0.8835 - precision: 0.9295 - recall: 0.8300 - TP: 830.0000 - TN: 937.0000 - FP: 63.0000 - FN: 170.0000
[0.28510403633117676, 0.8834999799728394, 0.9294512867927551, 0.8299999833106995, 830.0, 937.0, 63.0, 170.0, 0.8769149400548438, 0.8464317976513098, 0.937, 0.8894162316089226]
[00:34:34] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.903, 0.899009900990099, 0.908, 0.9034825870646767, 0.907070707070707, 0.898, 0.9025125628140703]
[0.9095, 0.9026548672566371, 0.918, 0.9102627664848786, 0.9165818921668362, 0.901, 0.908724155320222]
               loss   accuracy  precision     recall          TP          TN  \
count     10.000000  10.000000  10.000000  10.000000   10.000000   10.000000   
mean       0.246475   0.

### Split by Country - Shuffled

In [117]:
train_index = [val_split for _, val_split in skf_train.split(X_train,y_train)] 
val_index = [val_split for _, val_split in skf_val.split(X_val,y_val)] 

test_result, test_result_xgb, test_result_svm = run_test(X_train, y_train, X_val, y_val,
                       train_index, val_index, class_weight = {0: 1., 1: 1.}, epochs=1)
print(test_result)
print(test_result_xgb)
print(test_result_svm)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 17s 265ms/step - loss: 0.8276 - accuracy: 0.7080 - precision: 0.6525 - recall: 0.8900 - TP: 890.0000 - TN: 526.0000 - FP: 474.0000 - FN: 110.0000
[0.8275672793388367, 0.7080000042915344, 0.6524926424026489, 0.8899999856948853, 890.0, 526.0, 474.0, 110.0, 0.7529610603333862, 0.8270440251572327, 0.526, 0.6430317848410758]
[02:51:26] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.688, 0.6525974025974026, 0.804, 0.7204301075268817, 0.7447916666666666, 0.572, 0.6470588235294118]
[0.6935, 0.6455981941309256, 0.858, 0.7367969085444397, 0.7883755588673621, 0.529, 0.6331538001196888]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 253ms/step - loss: 0.6795 - accuracy: 0.7185 - precision: 0.6504 - recall: 0.9450 - TP: 945.0000 - TN: 492.0000 - FP: 508.0000 - FN: 55.0000
[0.6795116662979126, 0.718500018119812, 0.6503785252571106, 0.9449999928474426, 945.0, 492.0, 508.0, 55.0, 0.7704851165305981, 0.8994515539305301, 0.492, 0.6360698125404007]
[02:57:05] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.715, 0.6771004942339374, 0.822, 0.7425474254742548, 0.7735368956743003, 0.608, 0.6808510638297873]
[0.716, 0.6653905053598775, 0.869, 0.753686036426713, 0.8112391930835735, 0.563, 0.6646989374262101]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

63/63 [==============================] - 16s 260ms/step - loss: 0.7860 - accuracy: 0.7455 - precision: 0.6905 - recall: 0.8900 - TP: 890.0000 - TN: 601.0000 - FP: 399.0000 - FN: 110.0000
[0.7860105633735657, 0.7455000281333923, 0.6904577016830444, 0.8899999856948853, 890.0, 601.0, 399.0, 110.0, 0.7776321372327734, 0.8452883263009845, 0.601, 0.7025131502045587]
[03:30:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.726, 0.6810897435897436, 0.85, 0.7562277580071174, 0.800531914893617, 0.602, 0.687214611872146]
[0.7415, 0.6906077348066298, 0.875, 0.771945302161447, 0.8294679399727148, 0.608, 0.701673398730525]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 15s 241ms/step - loss: 1.0115 - accuracy: 0.6715 - precision: 0.6197 - recall: 0.8880 - TP: 888.0000 - TN: 455.0000 - FP: 545.0000 - FN: 112.0000
[1.0114799737930298, 0.671500027179718, 0.6196789741516113, 0.8880000114440918, 888.0, 455.0, 545.0, 112.0, 0.7299629979532722, 0.8024691358024691, 0.455, 0.5807275047862157]
[03:36:21] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.667, 0.6235207100591716, 0.843, 0.7168367346938775, 0.7577160493827161, 0.491, 0.595873786407767]
[0.6755, 0.6306775874906925, 0.847, 0.7230046948356808, 0.7671232876712328, 0.504, 0.6083283041641521]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


63/63 [==============================] - 16s 257ms/step - loss: 0.8217 - accuracy: 0.7215 - precision: 0.6627 - recall: 0.9020 - TP: 902.0000 - TN: 541.0000 - FP: 459.0000 - FN: 98.0000
[0.8217496275901794, 0.7214999794960022, 0.6627479791641235, 0.9020000100135803, 902.0, 541.0, 459.0, 98.0, 0.7640830190894456, 0.8466353677621283, 0.541, 0.6601586333129958]
[03:41:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.6975, 0.6488319517709118, 0.861, 0.740008594757198, 0.7934621099554234, 0.534, 0.638374178123132]
[0.707, 0.6556390977443609, 0.872, 0.7484978540772533, 0.808955223880597, 0.542, 0.6491017964071857]
               loss   accuracy  precision     recall          TP          TN  \
count     10.000000  10.000000  10.000000  10.000000   10.000000   10.000000   
mean       0.777301   0.7

### Country Balanced Split - Shuffled

In [118]:
# Load in data
train_count = 0  
val_count = 0  
train_label_counter = [0, 0]
val_label_counter = [0, 0]

train_label_count_max = 5e3 
val_label_count_max = 1e3
total_train = 2 * train_label_count_max
total_val = 2 * val_label_count_max

X_train = []
y_train = []

X_val = []
y_val = []

for line in fs.open('s3://compressed-data-sample/shuffled_train.json'):
    if train_count >= total_train and val_count >= total_val:
        break
    json_file = json.loads(line)
    country = json_file['country']
    label =  int(json_file['country'] in PEACE_COUNTRY)
    
    if not country in MAJOR_COUNTRY:
        if train_label_counter[label] < train_label_count_max :
            sent = json_file['content_cleaned_shuffled']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_train.append({'input_ids': ids,'attention_mask':msk})
            y_train.append(json_file['country'])
            train_count += 1
            train_label_counter[label] += 1
            if sum(train_label_counter) % 1e3 == 0:
                print('Train', train_label_counter)
    else:
        if val_label_counter[label] < val_label_count_max :
            sent = json_file['content_cleaned_shuffled']
            ids, msk = regular_encode(sent, tokenizer) # tokenize content_cleaned
            
            X_val.append({'input_ids': ids,'attention_mask':msk})
            y_val.append(json_file['country'])
            val_count += 1
            val_label_counter[label] += 1
            if sum(val_label_counter) % 1e3 == 0:
                print('Val', val_label_counter)

Val [532, 468]
Val [1000, 1000]
Train [240, 760]
Train [481, 1519]
Train [749, 2251]
Train [1028, 2972]
Train [1268, 3732]
Train [1522, 4478]
Train [2000, 5000]
Train [3000, 5000]
Train [4000, 5000]
Train [5000, 5000]


In [121]:
median = statistics.median(list(Counter(y_train).values()))
median

287

In [124]:
## Prepare K-Fold for minority country
X_train = np.array(X_train)
X_val = np.array(X_val)

y_train = np.array(y_train)
y_val = np.array(y_val)

australia_idx = np.where(np.array(y_val) == 'Australia')[0]
india_idx = np.where(np.array(y_val) == 'India')[0]
majority_index = []
for i in range(10):
    australia_sample = np.array(random.sample(list(australia_idx), median))
    india_sample = np.array(random.sample(list(india_idx), median))
    majority_index.append(np.hstack([australia_sample, india_sample]))   

In [125]:
eval_results = []
eval_results_xgb = []
eval_results_svm = []
metrics_names = None
    
for maj_index in majority_index:
    tf.keras.backend.clear_session()
    X_all_sample = np.hstack([X_train, X_val[maj_index]])
    y_all_sample = np.hstack([y_train, y_val[maj_index]])
    y_all_sample = np.array(list(map(lambda x: int(x in PEACE_COUNTRY), y_all_sample)))
    
    X_train_fold, X_val_fold, y_train_fold, y_val_fold = train_test_split(X_all_sample, y_all_sample, test_size=0.2)
    
    model = get_model()
        
    train_input1 = np.vstack([x['input_ids'] for x in X_train_fold])
    train_input2 = np.vstack([x['attention_mask'] for x in X_train_fold])
    model.fit(x=[train_input1, train_input2], 
              y=np.asarray(y_train_fold),
              epochs = 1, 
              batch_size = 32,
              class_weight={0: 1., 1: 1.})

    eval_input1 = np.vstack([x['input_ids'] for x in X_val_fold])
    eval_input2 = np.vstack([x['attention_mask'] for x in X_val_fold])
    er = model.evaluate(x=[eval_input1, eval_input2], 
                        y=np.asarray(y_val_fold), return_dict=True)
    f1 = 2*er['precision']*er['recall'] / (er['precision']+er['recall'])

    precision_neg = er['TN'] / (er['TN'] + er['FN'])
    recall_neg = er['TN'] / (er['TN']+er['FP'])
    f1_neg = 2 * precision_neg * recall_neg / (precision_neg + recall_neg)

    er = list(er.values())
    er += [f1, precision_neg, recall_neg, f1_neg]
    eval_results.append(er)
    metrics_names = model.metrics_names
    print(er)

    # Get Embeddings 
    input_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='input_ids', dtype='int32')
    input_masks_ids_in = tf.keras.layers.Input(shape=(MAX_LEN,), name='attention_mask', dtype='int32')
    output_layer = model.layers[2].layers[0](input_ids_in, input_masks_ids_in)[0]
    embedding_model = tf.keras.Model(inputs=[input_ids_in, input_masks_ids_in], outputs = output_layer)

    train_embedding = embedding_model.predict(x=[train_input1, train_input2])[:, 0]
    val_embedding = embedding_model.predict(x=[eval_input1, eval_input2])[:, 0]

    # Get XGB results
    dtrain = xgb.DMatrix(train_embedding, label=y_train_fold)
    dval = xgb.DMatrix(val_embedding, label=y_val_fold)
    param = {'max_depth': 2, 'eta': 1,'objective': 'binary:logistic'}
    num_round = 10
    evallist = [(dval, 'eval'), (dtrain, 'train')]
    bst = xgb.train(param, dtrain, num_round, evallist, verbose_eval=False)
    y_pred = (bst.predict(dval) > 0.5).astype(int)
    xgb_acc = np.mean(y_val_fold == y_pred)
    xgb_score = precision_recall_fscore_support(y_val_fold, y_pred)
    # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
    er = [xgb_acc, xgb_score[0][1], xgb_score[1][1], xgb_score[2][1], 
                             xgb_score[0][0], xgb_score[1][0], xgb_score[2][0]]
    print(er)
    eval_results_xgb.append(er)


    # Get SVM results
    pipe = Pipeline([('scaler', StandardScaler()),
                     ('pca', PCA()),
                     ('svc', SVC())])
    pipe.fit(train_embedding, y_train_fold)
    y_pred = (pipe.predict(val_embedding) > 0.5).astype(int)
    svm_acc = np.mean(y_val_fold == y_pred)
    svm_score = precision_recall_fscore_support(y_val_fold, y_pred)
    # Accuracy, Precision+, Recall+, F1+, Precision-, Recall-, F1-
    er = [svm_acc, svm_score[0][1], svm_score[1][1], svm_score[2][1], 
                             svm_score[0][0], svm_score[1][0], svm_score[2][0]]
    print(er)
    eval_results_svm.append(er)
    
eval_results = np.array(eval_results)
metrics_names += ['f1', 'precision_neg', 'recall_neg', 'f1_neg']
eval_results = pd.DataFrame(eval_results, columns=metrics_names)
ci = eval_results.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results = pd.concat([eval_results.describe(), ci])

eval_results_xgb = np.array(eval_results_xgb)
eval_results_xgb = pd.DataFrame(eval_results_xgb, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
ci = eval_results_xgb.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results_xgb = pd.concat([eval_results_xgb.describe(), ci])

eval_results_svm = np.array(eval_results_svm)
eval_results_svm = pd.DataFrame(eval_results_svm, columns=['accuracy', 'precision', 'recall', 'f1', 'precision_neg', 'recall_neg', 'f1_neg'])
ci = eval_results_svm.apply(lambda x: st.t.interval(0.95, len(x), loc=np.mean(x), scale=st.sem(x)))
ci.index = ['ci_lower', 'ci_upper']
eval_results_svm = pd.concat([eval_results_svm.describe(), ci])

eval_results, eval_results_xgb, eval_results_svm

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 18s 265ms/step - loss: 0.2325 - accuracy: 0.9106 - precision: 0.9311 - recall: 0.8883 - TP: 946.0000 - TN: 980.0000 - FP: 70.0000 - FN: 119.0000
[0.2325342297554016, 0.9106382727622986, 0.9311023354530334, 0.888262927532196, 946.0, 980.0, 70.0, 119.0, 0.90917827568558, 0.89171974522293, 0.9333333333333333, 0.9120521172638437]
[03:58:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9125295508274232, 0.9104477611940298, 0.9164319248826291, 0.9134300421151147, 0.9146692233940557, 0.9085714285714286, 0.9116101290014333]
[0.915839243498818, 0.9140989729225023, 0.9192488262910798, 0.9166666666666666, 0.9176245210727969, 0.9123809523809524, 0.9149952244508118]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 258ms/step - loss: 0.2159 - accuracy: 0.9158 - precision: 0.9091 - recall: 0.9229 - TP: 970.0000 - TN: 967.0000 - FP: 97.0000 - FN: 81.0000
[0.2158750742673874, 0.9158392548561096, 0.9090909361839294, 0.9229305386543274, 970.0, 967.0, 97.0, 81.0, 0.9159584633057599, 0.9227099236641222, 0.9088345864661654, 0.915719696969697]
[04:04:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9106382978723404, 0.9058380414312618, 0.9153187440532826, 0.9105537150970185, 0.9154795821462488, 0.9060150375939849, 0.9107227208313652]
[0.91725768321513, 0.9040590405904059, 0.9324452901998097, 0.9180327868852459, 0.9311348205625606, 0.9022556390977443, 0.9164677804295942]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 256ms/step - loss: 0.1943 - accuracy: 0.9187 - precision: 0.9161 - recall: 0.9161 - TP: 939.0000 - TN: 1004.0000 - FP: 86.0000 - FN: 86.0000
[0.1943126618862152, 0.9186761379241943, 0.9160975813865662, 0.9160975813865662, 939.0, 1004.0, 86.0, 86.0, 0.9160975813865662, 0.9211009174311927, 0.9211009174311927, 0.9211009174311927]
[04:46:17] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9139479905437352, 0.9096209912536443, 0.9131707317073171, 0.9113924050632911, 0.91804788213628, 0.9146788990825688, 0.9163602941176471]
[0.9295508274231679, 0.9227799227799228, 0.9326829268292683, 0.9277049975739933, 0.9360518999073216, 0.926605504587156, 0.9313047487321346]


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


67/67 [==============================] - 17s 255ms/step - loss: 0.1845 - accuracy: 0.9225 - precision: 0.9260 - recall: 0.9208 - TP: 988.0000 - TN: 963.0000 - FP: 79.0000 - FN: 85.0000
[0.1844778060913086, 0.9224586486816406, 0.9259606599807739, 0.9207828640937805, 988.0, 963.0, 79.0, 85.0, 0.9233645034304653, 0.9188931297709924, 0.9241842610364683, 0.9215311004784689]
[04:52:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.9224586288416076, 0.9275634995296331, 0.918918918918919, 0.9232209737827715, 0.9173003802281369, 0.9261036468330134, 0.9216809933142311]
[0.9271867612293144, 0.9282385834109972, 0.9282385834109972, 0.9282385834109972, 0.9261036468330134, 0.9261036468330134, 0.9261036468330134]


(               loss   accuracy  precision     recall           TP  \
 count     10.000000  10.000000  10.000000  10.000000    10.000000   
 mean       0.206965   0.915745   0.920650   0.909532   958.000000   
 std        0.013875   0.006220   0.015859   0.020139    28.944391   
 min        0.184478   0.906856   0.896190   0.883165   924.000000   
 25%        0.196977   0.910638   0.910758   0.890034   939.500000   
 50%        0.209301   0.916785   0.918563   0.915289   953.000000   
 75%        0.213372   0.920449   0.929532   0.922394   969.500000   
 max        0.232534   0.924823   0.953327   0.941667  1017.000000   
 ci_lower   0.197188   0.911362   0.909476   0.895342   937.605800   
 ci_upper   0.216741   0.920127   0.931825   0.923723   978.394200   
 
                    TN          FP          FN         f1  precision_neg  \
 count       10.000000   10.000000   10.000000  10.000000      10.000000   
 mean       978.800000   82.900000   95.300000   0.914791       0.911748   
